**KoELECTRA를 활용한 한국어 감정분석 모델**

본 한국어 감정분석 모델은 고려대학교 디지털금융공학과 DFE610 디지털금융공학을 위한 자연어처리기술 수업의 과제를 수행하기 위해 개발한 모델이다.

본 한국어 감정분석 모델은 '네이버 영화리뷰 감정분석 with Hugging Face BERT'의 오픈소스를 참고하여 구성하였다.


1.   모델의 오픈소스 참고 출처

     @ Transformer를 이용한 감정분석 모델 : https://github.com/Parkchanjun/KU-NLP-2020-1/Transformer를 이용한 감정분석_한국어.jpynb


2. 상세사항
- 아래 깃허브의 Readme.md 확인

  https://github.com/yklove1025/Ensamble-Sentiment-Analysis-for-Korean

3. 실행환경
- 본 소스는 구글 Colab에서 작성됨 
- 본 소스의 실행을 위해서는 Colab Pro 환경이 필요(GPU 메모리 필요) 


**Hugging Face의 Transformer Model 설치 ①**

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 13.3MB/s 
     |████████████████████████████████| 890kB 56.1MB/s 
     |████████████████████████████████| 2.9MB 58.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=77f85f03a6392d10516dc54351d23aa59dd7e87a0230d1cffad6b643d9afc485
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


**학습 데이터 다운로드 (Naver, NSMC) ①**

In [ ]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 18.34 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


**공통 패키지 Import ①**

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
import torch
import tensorflow as tf
import keras
import os
import nltk
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from keras.models import Sequential
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from transformers import ElectraTokenizer
from transformers import ElectraForSequenceClassification, AdamW

**데이터 로드 ①** 

In [ ]:
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


- Lable data set ①,②

In [ ]:
labels_train = train['label'].values
labels_test = test['label'].values

**KoELECTRA 모델 텍스트 전처리 ①**

In [ ]:
sentences = train['document']

X_train_bert = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
X_train_bert[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]']

In [ ]:
sentences = test['document']

X_test_bert = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
X_test_bert[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] GDNTOPCLASSINTHECLUB [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? [SEP]',
 '[CLS] 음악이 주가 된, 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. [SEP]']

**테스트를 위한 모델 정확도 산출 함수 선언(1만개 테스트 데이터 대상) ①**

In [ ]:
def convert_acc (x) :

    acc = []

    for i, row in x.iterrows() :
      if x.at[i, 'Predicted'] == x.at[i, 'label'] : i = 1 
      else : i = 0
      acc.append(i)   

    return acc

In [ ]:
labels_test_input = labels_test[:10000]

**KoELECTRA 감정분석 데이터 처리 ①**

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator", do_lower_case=False)

X_train_tokenized_text = [tokenizer.tokenize(sent) for sent in X_train_bert]
X_test_tokenized_text = [tokenizer.tokenize(sent) for sent in X_test_bert]

print (X_train_tokenized_text[0])
print (X_test_tokenized_text[0])

# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
X_train_bert = [tokenizer.convert_tokens_to_ids(x) for x in X_train_tokenized_text]
X_test_bert = [tokenizer.convert_tokens_to_ids(x) for x in X_test_tokenized_text]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
X_train_bert = pad_sequences(X_train_bert, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
X_test_bert = pad_sequences(X_test_bert, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


['[CLS]', '아', '더', '##빙', '.', '.', '진짜', '짜증', '##나', '##네', '##요', '목소리', '[SEP]']
['[CLS]', '굳', '[UNK]', '[SEP]']


**KoELECTRA 감정분석 모델 설정 ①**

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in X_train_bert:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(X_train_bert,
                                                                                    labels_train, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       X_train_bert,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

attention_masks = []

for seq in X_test_bert:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(X_test_bert)
test_labels = torch.tensor(labels_test)
test_masks = torch.tensor(attention_masks)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
# 분류를 위한 BERT 모델 생성
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v2-discriminator", num_labels = 2)
model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 5

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

**KoELECTRA 감정분석 모델 학습 실행 ①**

- Training, Validation 진행 (Colab Out of memory 시 초기화 후 분할 재실행 필요)

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:02:00.
  Batch 1,000  of  4,219.    Elapsed: 0:03:59.
  Batch 1,500  of  4,219.    Elapsed: 0:05:59.
  Batch 2,000  of  4,219.    Elapsed: 0:07:59.
  Batch 2,500  of  4,219.    Elapsed: 0:09:59.
  Batch 3,000  of  4,219.    Elapsed: 0:11:58.
  Batch 3,500  of  4,219.    Elapsed: 0:13:58.
  Batch 4,000  of  4,219.    Elapsed: 0:15:57.

  Average training loss: 0.30
  Training epcoh took: 0:16:50

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:32

======== Epoch 2 / 5 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:02:00.
  Batch 1,000  of  4,219.    Elapsed: 0:04:00.
  Batch 1,500  of  4,219.    Elapsed: 0:06:00.
  Batch 2,000  of  4,219.    Elapsed: 0:07:59.
  Batch 2,500  of  4,219.    Elapsed: 0:09:59.
  Batch 3,000  of  4,219.    Elapsed: 0:11:59.
  Batch 3,500  of  4,219.    Elapsed: 0:13:59.
  Batch 4,000  of  4,219.    Elapsed: 0:15:59.

  Average training loss: 0.22
  

**BERT 감정분석 모델 테스트 ①,②**

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    BERT_output = []
    BERT_output = BERT_output.append(logits)
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:07.
  Batch   200  of  1,563.    Elapsed: 0:00:13.
  Batch   300  of  1,563.    Elapsed: 0:00:20.
  Batch   400  of  1,563.    Elapsed: 0:00:27.
  Batch   500  of  1,563.    Elapsed: 0:00:34.
  Batch   600  of  1,563.    Elapsed: 0:00:40.
  Batch   700  of  1,563.    Elapsed: 0:00:47.
  Batch   800  of  1,563.    Elapsed: 0:00:54.
  Batch   900  of  1,563.    Elapsed: 0:01:01.
  Batch 1,000  of  1,563.    Elapsed: 0:01:07.
  Batch 1,100  of  1,563.    Elapsed: 0:01:14.
  Batch 1,200  of  1,563.    Elapsed: 0:01:21.
  Batch 1,300  of  1,563.    Elapsed: 0:01:28.
  Batch 1,400  of  1,563.    Elapsed: 0:01:34.
  Batch 1,500  of  1,563.    Elapsed: 0:01:41.

Accuracy: 0.90
Test took: 0:01:45


**BERT 감정분석 모델 신규 텍스트 함수 선언 ①,②**

In [ ]:
# 문장 테스트
def test_sentences(inputs, masks):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    #inputs, masks = convert_input_data(sentences)
    inputs = inputs
    masks = masks

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

**BERT를 이용한 감정분석 모델 테스트 Output 데이터 생성 ①,②** 

- 테스트 데이터 분리 (코랩 GPU 메모리 문제로 분할 실행)

In [ ]:
# Kaggle test 데이터 분리 (Colab GPU 활용 제한으로 분리하여 예측 결과 저장 후 병합)
test_inputs_1 = test_inputs[:2000]
test_masks_1 = test_masks[:2000]
test_inputs_2 = test_inputs[2000:4000]
test_masks_2 = test_masks[2000:4000]
test_inputs_3 = test_inputs[4000:6000]
test_masks_3 = test_masks[4000:6000]
test_inputs_4 = test_inputs[6000:8000]
test_masks_4 = test_masks[6000:8000]
test_inputs_5 = test_inputs[8000:10000]
test_masks_5 = test_masks[8000:10000]

- 테스트 데이터로 예측 수행 1

In [ ]:
logits = test_sentences(test_inputs_1, test_masks_1)

x_predict_BERT_1 = np.argmax(logits, axis = 1)
x_predict_BERT_1 = x_predict_BERT_1.tolist()
print('예측결과:', x_predict_BERT_1[:10])

예측결과: [1, 1, 0, 0, 0, 1, 0, 0, 0, 1]


- BERT output csv 파일 저장 1

In [ ]:
ensemble_BERT_1 = pd.DataFrame({'BERT prediction' : x_predict_BERT_1})
print(ensemble_BERT_1)

      BERT prediction
0                   1
1                   1
2                   0
3                   0
4                   0
...               ...
1995                0
1996                0
1997                0
1998                1
1999                0

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_1.to_csv('ensemble_BERT_1.csv')

- 테스트 데이터로 예측 수행 2

In [ ]:
logits = test_sentences(test_inputs_2, test_masks_2)

x_predict_BERT_2 = np.argmax(logits, axis = 1)
x_predict_BERT_2 = x_predict_BERT_2.tolist()
print('예측결과:', x_predict_BERT_2[:10])

예측결과: [1, 1, 1, 1, 0, 0, 1, 1, 0, 1]


- BERT output csv 파일 저장 2

In [ ]:
ensemble_BERT_2 = pd.DataFrame({'BERT prediction' : x_predict_BERT_2})
print(ensemble_BERT_2)

      BERT prediction
0                   1
1                   1
2                   1
3                   1
4                   0
...               ...
1995                0
1996                0
1997                0
1998                0
1999                1

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_2.to_csv('ensemble_BERT_2.csv')

- 테스트 데이터로 예측 수행 3

In [ ]:
logits = test_sentences(test_inputs_3, test_masks_3)

x_predict_BERT_3 = np.argmax(logits, axis = 1)
x_predict_BERT_3 = x_predict_BERT_3.tolist()
print('예측결과:', x_predict_BERT_3[:10])

예측결과: [1, 1, 1, 0, 0, 0, 0, 0, 1, 1]


- BERT output csv 파일 저장 3

In [ ]:
ensemble_BERT_3 = pd.DataFrame({'BERT prediction' : x_predict_BERT_3})
print(ensemble_BERT_3)

      BERT prediction
0                   1
1                   1
2                   1
3                   0
4                   0
...               ...
1995                0
1996                0
1997                1
1998                0
1999                0

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_3.to_csv('ensemble_BERT_3.csv')

- 테스트 데이터로 예측 수행 4

In [ ]:
logits = test_sentences(test_inputs_4, test_masks_4)

x_predict_BERT_4 = np.argmax(logits, axis = 1)
x_predict_BERT_4 = x_predict_BERT_4.tolist()
print('예측결과:', x_predict_BERT_4[:10])

예측결과: [0, 1, 0, 0, 1, 0, 0, 1, 1, 1]


- BERT output csv 파일 저장 4

In [ ]:
ensemble_BERT_4 = pd.DataFrame({'BERT prediction' : x_predict_BERT_4})
print(ensemble_BERT_4)

      BERT prediction
0                   0
1                   1
2                   0
3                   0
4                   1
...               ...
1995                1
1996                1
1997                0
1998                0
1999                0

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_4.to_csv('ensemble_BERT_4.csv')

- 테스트 데이터로 예측 수행 5

In [ ]:
logits = test_sentences(test_inputs_5, test_masks_5)

x_predict_BERT_5 = np.argmax(logits, axis = 1)
x_predict_BERT_5 = x_predict_BERT_5.tolist()
print('예측결과:', x_predict_BERT_5[:10])

예측결과: [0, 0, 1, 1, 0, 0, 0, 1, 0, 0]


- BERT output csv 파일 저장 5

In [ ]:
ensemble_BERT_5 = pd.DataFrame({'BERT prediction' : x_predict_BERT_5})
print(ensemble_BERT_5)

      BERT prediction
0                   0
1                   0
2                   1
3                   1
4                   0
...               ...
1995                1
1996                0
1997                1
1998                0
1999                0

[2000 rows x 1 columns]


In [ ]:
ensemble_BERT_5.to_csv('ensemble_BERT_5.csv')

**KoELECTRA 예측결과 CSV 파일 저장**

In [ ]:
KoELECTRA_kor = np.concatenate((x_predict_BERT_1, x_predict_BERT_2, x_predict_BERT_3, x_predict_BERT_4, x_predict_BERT_5), axis = 0)

In [ ]:
KoELECTRA_kor_out = pd.DataFrame({'Predicted' : KoELECTRA_kor})
print(KoELECTRA_kor_out)

      Predicted
0             1
1             1
2             0
3             0
4             0
...         ...
9995          1
9996          0
9997          1
9998          0
9999          0

[10000 rows x 1 columns]


In [ ]:
KoELECTRA_kor_out.to_csv('KoELECTRA_kor.csv')

- KoELECTRA 모델 예측 결과 스코어링

In [ ]:
labels_test_input = labels_test[:10000]

In [ ]:
print("KoELECTRA model accuracy : ", accuracy_score(labels_test_input, KoELECTRA_kor_out))
print("KoELECTRA model recall : ",recall_score(labels_test_input, KoELECTRA_kor_out))
print("KoELECTRA model precision : ",precision_score(labels_test_input, KoELECTRA_kor_out))
print("KoELECTRA model f1 : ",f1_score(labels_test_input, KoELECTRA_kor_out))

KoELECTRA model accuracy :  0.8962
KoELECTRA model recall :  0.9025884212606204
KoELECTRA model precision :  0.8934089575591629
KoELECTRA model f1 :  0.8979752309809318
